In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -r requirement.txt

In [3]:
import torch

from collections import Counter
from utils.box.bbox import decode
from utils.box.rbbox import rbbox_batched_nms as nms

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('60000_wo_backbone.csv')

In [6]:
df

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ05286,0.999829,604.8,463.2,622.0,472.6,607.6,499.2,590.4,489.9
1,OBJ05286,0.997913,557.2,430.5,578.9,444.9,561.6,471.0,539.9,456.6
2,OBJ05286,0.994679,812.8,740.9,869.7,742.6,869.0,766.1,812.1,764.3
3,OBJ05286,0.982474,560.2,434.4,570.7,441.1,553.8,467.6,543.3,461.0
4,OBJ05286,0.974291,649.0,494.2,665.9,503.3,661.8,511.0,644.8,502.0
...,...,...,...,...,...,...,...,...,...,...
8940,OBJ04116,0.119079,774.5,691.0,799.8,697.7,798.7,701.9,773.4,695.2
8941,OBJ04116,0.115704,544.5,253.7,569.1,254.6,568.8,261.7,544.2,260.8
8942,OBJ04116,0.109384,810.6,824.4,773.0,824.6,772.9,814.9,810.5,814.7
8943,OBJ04116,0.105901,423.3,368.3,435.3,368.6,435.0,382.0,423.0,381.7


In [7]:
import sys

sys.path.append('.')

import os
import tqdm
import torch
import numpy as np

from collections import defaultdict
from torch.utils.data import DataLoader

from data.aug import ops
from data.aug.compose import Compose
from data.dataset import DOTA

from model.rdd import RDD
from model.backbone import resnet

from utils.box.bbox_np import xywha2xy4, xy42xywha
from utils.box.rbbox_np import rbbox_batched_nms
from utils.parallel import CustomDetDataParallel

In [8]:
ret_raw = {}

for i in range(len(df)):
  X = df.iloc[i]
  fname = X['File']
  if fname not in ret_raw:
    ret_raw[fname] = []
  scores = np.array([X['Confidence']])
  xy4 = np.array([[X['X1'], X['Y1']], [X['X2'], X['Y2']], [X['X3'], X['Y3']], [X['X4'], X['Y4']]])
  xywha = xy42xywha(xy4)
  # xy4 = np.array([[X['X1'], X['X2'], X['X3'], X['X4']], [X['Y1'], X['Y2'], X['Y3'], X['Y4']]])
  ret_raw[fname].append([xywha, scores, np.array([0])])

In [19]:

ret = []
nms_thresh = 0.3
for fname, value in ret_raw.items():
  bboxes, scores, labels = zip(*value)
  bboxes = np.concatenate(list(bboxes)).reshape(-1, 5)
  scores = np.concatenate(list(scores))
  labels = np.concatenate(list(labels))
  keeps = rbbox_batched_nms(bboxes, scores, labels, nms_thresh)
  ret.append([fname, [bboxes[keeps], scores[keeps], labels[keeps]]])

In [20]:
print('converting to submission format...')
ret_save = defaultdict(list)
for fname, (bboxes, scores, labels) in ret:
    for bbox, score, label in zip(bboxes, scores, labels):
        bbox = xywha2xy4(bbox).ravel()
        line = '%s %.12f %.1f %.1f %.1f %.1f %.1f %.1f %.1f %.1f' % (fname, score, *bbox)
        ret_save['container'].append(line)

converting to submission format...


In [21]:
ret_save

defaultdict(list,
            {'container': ['OBJ05286 0.999829411507 604.8 463.2 622.0 472.5 607.6 499.2 590.4 489.9',
              'OBJ05286 0.997912943363 557.2 430.5 578.9 444.9 561.6 471.0 539.9 456.6',
              'OBJ05286 0.994678616524 812.8 740.9 869.7 742.6 869.0 766.0 812.1 764.3',
              'OBJ05286 0.974290728569 649.0 494.2 665.9 503.3 661.7 511.1 644.8 502.0',
              'OBJ05286 0.957170307636 500.6 132.2 531.9 144.8 529.6 150.7 498.3 138.1',
              'OBJ05286 0.930917739868 139.2 1.9 167.8 4.6 166.6 17.0 138.0 14.3',
              'OBJ05286 0.850561141968 738.5 473.3 743.8 476.6 729.6 499.6 724.3 496.3',
              'OBJ05286 0.850545406342 559.3 898.8 578.1 912.9 574.2 918.1 555.4 904.0',
              'OBJ05286 0.843254745007 386.2 25.5 417.6 45.8 405.4 64.7 374.0 44.4',
              'OBJ05286 0.703096449375 550.3 423.4 555.9 427.2 541.1 449.0 535.5 445.2',
              'OBJ05286 0.589792132378 822.2 524.9 831.8 531.8 814.5 556.0 804.9 549.1',


In [22]:
print('saving...')
dir_save = '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/'
os.makedirs(os.path.join(dir_save, 'submission'), exist_ok=True)
for name, dets in ret_save.items():
    with open(os.path.join(dir_save, 'submission', 'Task%d_%s.txt' % (1, name)), 'wt') as f:
        f.write('\n'.join(dets))

print('finished')

saving...
finished


In [23]:
len(df), len(ret_save['container'])

(8945, 7375)

In [73]:
tmp = ret_save['container']
data = [tmp[i].split(' ') for i in range(len(tmp))]
print(data)

[['OBJ05286', '0.999829411507', '604.8', '463.2', '622.0', '472.5', '607.6', '499.2', '590.4', '489.9'], ['OBJ05286', '0.997912943363', '557.2', '430.5', '578.9', '444.9', '561.6', '471.0', '539.9', '456.6'], ['OBJ05286', '0.994678616524', '812.8', '740.9', '869.7', '742.6', '869.0', '766.0', '812.1', '764.3'], ['OBJ05286', '0.974290728569', '649.0', '494.2', '665.9', '503.3', '661.7', '511.1', '644.8', '502.0'], ['OBJ05286', '0.957170307636', '500.6', '132.2', '531.9', '144.8', '529.6', '150.7', '498.3', '138.1'], ['OBJ05286', '0.930917739868', '139.2', '1.9', '167.8', '4.6', '166.6', '17.0', '138.0', '14.3'], ['OBJ05286', '0.850561141968', '738.5', '473.3', '743.8', '476.6', '729.6', '499.6', '724.3', '496.3'], ['OBJ05286', '0.850545406342', '559.3', '898.8', '578.1', '912.9', '574.2', '918.1', '555.4', '904.0'], ['OBJ05286', '0.843254745007', '386.2', '25.5', '417.6', '45.8', '405.4', '64.7', '374.0', '44.4'], ['OBJ05286', '0.703096449375', '550.3', '423.4', '555.9', '427.2', '541.1

In [77]:
df2 = pd.DataFrame(data)
df2.columns = df.columns
df2

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ05286,0.999829411507,604.8,463.2,622.0,472.5,607.6,499.2,590.4,489.9
1,OBJ05286,0.997912943363,557.2,430.5,578.9,444.9,561.6,471.0,539.9,456.6
2,OBJ05286,0.994678616524,812.8,740.9,869.7,742.6,869.0,766.0,812.1,764.3
3,OBJ05286,0.974290728569,649.0,494.2,665.9,503.3,661.7,511.1,644.8,502.0
4,OBJ05286,0.957170307636,500.6,132.2,531.9,144.8,529.6,150.7,498.3,138.1
...,...,...,...,...,...,...,...,...,...,...
7370,OBJ04116,0.119078516960,774.5,691.0,799.8,697.7,798.7,701.9,773.4,695.2
7371,OBJ04116,0.115703687072,544.5,253.7,569.1,254.6,568.8,261.7,544.2,260.8
7372,OBJ04116,0.109383679926,810.6,824.4,773.0,824.6,772.9,814.9,810.5,814.7
7373,OBJ04116,0.105901300907,423.3,368.3,435.3,368.6,435.0,382.0,423.0,381.7


In [84]:
X = df2[df2['File'] == 'OBJ05286']
Y = df[df['File'] == 'OBJ05286']
len(X), len(Y)

(36, 39)

In [89]:
Y

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ05286,0.999829,604.8,463.2,622.0,472.6,607.6,499.2,590.4,489.9
1,OBJ05286,0.997913,557.2,430.5,578.9,444.9,561.6,471.0,539.9,456.6
2,OBJ05286,0.994679,812.8,740.9,869.7,742.6,869.0,766.1,812.1,764.3
3,OBJ05286,0.982474,560.2,434.4,570.7,441.1,553.8,467.6,543.3,461.0
4,OBJ05286,0.974291,649.0,494.2,665.9,503.3,661.8,511.0,644.8,502.0
5,OBJ05286,0.957170,500.6,132.2,531.9,144.8,529.6,150.6,498.3,138.1
6,OBJ05286,0.930918,139.2,1.9,167.8,4.6,166.6,17.0,138.0,14.3
7,OBJ05286,0.850561,738.5,473.3,743.8,476.6,729.7,499.6,724.3,496.3
8,OBJ05286,0.850545,559.3,898.8,578.1,912.9,574.2,918.0,555.5,904.0
9,OBJ05286,0.843255,386.2,25.5,417.6,45.8,405.4,64.7,374.0,44.4


In [85]:
X

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ05286,0.999829411507,604.8,463.2,622.0,472.5,607.6,499.2,590.4,489.9
1,OBJ05286,0.997912943363,557.2,430.5,578.9,444.9,561.6,471.0,539.9,456.6
2,OBJ05286,0.994678616524,812.8,740.9,869.7,742.6,869.0,766.0,812.1,764.3
3,OBJ05286,0.974290728569,649.0,494.2,665.9,503.3,661.7,511.1,644.8,502.0
4,OBJ05286,0.957170307636,500.6,132.2,531.9,144.8,529.6,150.7,498.3,138.1
5,OBJ05286,0.930917739868,139.2,1.9,167.8,4.6,166.6,17.0,138.0,14.3
6,OBJ05286,0.850561141968,738.5,473.3,743.8,476.6,729.6,499.6,724.3,496.3
7,OBJ05286,0.850545406342,559.3,898.8,578.1,912.9,574.2,918.1,555.4,904.0
8,OBJ05286,0.843254745007,386.2,25.5,417.6,45.8,405.4,64.7,374.0,44.4
9,OBJ05286,0.703096449375,550.3,423.4,555.9,427.2,541.1,449.0,535.5,445.2


In [28]:
cd '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/run/dota'

/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/run/dota


In [52]:
! chmod 777 '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images'

In [ ]:
# ! python visualize_label.py --image "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images" --label "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/60000_wo_backbone.csv"
! python visualize_label.py --image "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images" --label "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/submission/Task1_container.txt"